# Exploring New York Times blog dataset

The goal of this notebook is perform the initial analysis of the New York Time blog dataset.

## Loading necessary libraries

In [1]:
import pandas as pd # Pandas for Data Frame 
import numpy as np # NumPy for vetorial operations

## Loading the dataset

In [2]:
# Reads the CSV file using Pandas
dataset = pd.read_csv("NYTimesBlogTrain.csv")

## Presenting the first 5 rows

This initial view shows the attributes available and their content. As the table shows, there is at least one column with missing values.

In [3]:
# Presents the top 5 rows
dataset.head()

,NewsDesk,SectionName,SubsectionName,Headline,Snippet,Abstract,WordCount,PubDate,Popular,UniqueID
0,Business,Crosswords/Games,NaN,More School Daze,A puzzle from Ethan Cooper that reminds me tha...,A puzzle from Ethan Cooper that reminds me tha...,508,2014-09-01 22:00:09,1,1
1,Culture,Arts,NaN,New 96-Page Murakami Work Coming in December,The Strange Library will arrive just three and...,The Strange Library will arrive just three and...,285,2014-09-01 21:14:07,0,2
2,Business,Business Day,Dealbook,Public Pension Funds Stay Mum on Corporate Expats,Public pension funds have major stakes in Amer...,Public pension funds have major stakes in Amer...,1211,2014-09-01 21:05:36,0,3
3,Business,Business Day,Dealbook,Boot Camp for Bankers,As they struggle to find new business to bolst...,As they struggle to find new business to bolst...,1405,2014-09-01 20:43:34,1,4
4,Science,Health,NaN,Of Little Help to Older Knees,Middle-aged and older patients are unlikely to...,Middle-aged and older patients are unlikely to...,181,2014-09-01 18:58:51,1,5


## Identifying the period in which the news were recorded

By knowing the period, it is possible to estimate, for future steps, which time features could be used. By the fact that there are data corresponding to 3 months, it possible to discard month as a target for analysis, since there is not data for all months.

In [4]:
# Extracts the minimum and maximum date. By the fact that PubDate is organized in format: 
# YEAR-MONTH-DAY HOUR:MINUTE:SECOND, the lexicographic order corresponds to the chronological order.
"From {0} to {1}".format(dataset["PubDate"].min(), dataset["PubDate"].max())

'From 2014-09-01 00:01:32 to 2014-11-30 22:01:45'

## Selecting only columns of interest

For this initial analysis, only 4 variables will be analyzed.

In [5]:
# Creates a new dataframe using only specific columns
sub_dataset = dataset[["NewsDesk", "SectionName", "WordCount", "Popular"]]

## Counting the number of null values per column

Here we observe with detail for missing value on interest columns. As the results show, *NewsDesk* and *SectionName* contains missing values.

In [6]:
# A new data frame is created.For each attribute, the number of null values summed and results in a column
empty_values_summary = pd.DataFrame(
    {'NewsDesk': sub_dataset['NewsDesk'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'SectionName': sub_dataset['SectionName'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'WordCount': sub_dataset['WordCount'].isnull().apply(lambda x: 1 if x else 0).sum(),
     'Popular': sub_dataset['Popular'].isnull().apply(lambda x: 1 if x else 0).sum()},
      index=['Null Values'], columns=["NewsDesk", "SectionName", "WordCount", "Popular"])

# Presents the dataframe created
empty_values_summary

,NewsDesk,SectionName,WordCount,Popular
Null Values,1846,2300,0,0


## Summarizing the information about WordCount

The first variable analyzed is *WordCount*. The code below show statistics about the number of words from all publications. As it is possible to observe, there are publications without word and at least one with 10,912 words. In average, the number of words is 546.

In [7]:
# Select only WordCount and use describe function to extract the main statistics from values.
sub_dataset[['WordCount']].describe()

,WordCount
count,6532.000000
mean,524.434323
std,546.153272
min,0.000000
25%,187.000000
50%,374.000000
75%,723.250000
max,10912.000000


## Identifying the number of publications without words

To better estimate the number of publications without words, a specific count is performed.

In [8]:
# Selects only records with WordCount equals to 0 and count the number
sub_dataset[ sub_dataset['WordCount'] == 0 ]['WordCount'].count()

88

## Presenting the distribuition of NewsDesk values

*NewsDesk* is the category of a publication. The code below shows the percentage of publications for each *NewsDesk* value. The most frequent category is **Business**, while the less frequent is **Sports**.

In [9]:
sub_dataset['NewsDesk'].value_counts(sort=True, normalize=True)

Business    0.236987
TStyle      0.110839
Culture     0.103491
OpEd        0.079761
Foreign     0.057410
Styles      0.045468
Metro       0.030312
Science     0.029700
Travel      0.017759
Magazine    0.004746
National    0.000612
Sports      0.000306
dtype: float64

## Observing the distribuition of NewsDesk values by Popularity

The code above showed, proportionally, the number of publications by *NewsDesk*. Now we analyze, for each *NewsDesk*, the division between popular (indicated by *1*) and impopular (indicated by *0*) entries.

In [10]:
# Groups records by NewsDesk and Popular and counts the number of values
sub_dataset.groupby(["NewsDesk", "Popular"]).size()

NewsDesk  Popular
Business  0          1301
          1           247
Culture   0           626
          1            50
Foreign   0           372
          1             3
Magazine  0            31
Metro     0           181
          1            17
National  0             4
OpEd      0           113
          1           408
Science   0            73
          1           121
Sports    0             2
Styles    0           196
          1           101
TStyle    0           715
          1             9
Travel    0           115
          1             1
dtype: int64

## Observing the distribution of SectionName values by Popularity

In [11]:
# Groups the records by NewsDesk, SectionName and Popular and counts the number of values
sub_dataset.groupby(["NewsDesk", "SectionName", "Popular"]).size()

NewsDesk  SectionName       Popular
Business  Business Day      0          998
                            1           93
          Crosswords/Games  0           19
                            1          103
          Technology        0          280
                            1           50
Culture   Arts              0          625
                            1           50
Foreign   World             0          209
                            1            3
Magazine  Magazine          0           31
Metro     N.Y. / Region     0          181
                            1           17
National  U.S.              0            2
OpEd      Opinion           0          113
                            1          407
Science   Health            0           73
                            1          119
Sports    Sports            0            1
Styles    Health            0            1
          Style             0            2
          U.S.              0           77
                  

## Analyzing SectionNames distribution

In [12]:
sub_dataset['SectionName'].value_counts(sort=True, normalize=True)

Business Day        0.167177
Arts                0.103337
Opinion             0.092927
U.S.                0.077312
Technology          0.050521
World               0.032456
N.Y. / Region       0.030312
Health              0.029700
Multimedia          0.021586
Crosswords/Games    0.018830
Travel              0.017912
Magazine            0.004746
Open                0.000612
Style               0.000306
Sports              0.000153
dtype: float64

# Analyzing Popular distribution

In [13]:
sub_dataset['Popular'].value_counts(sort=True, normalize=True)

0    0.83267
1    0.16733
dtype: float64

## Summary

The variable analyzes showed us that:
 * **WordCount**: does not contain null values, but has 88 zero values. The values ranges from 0 to 10912 words. Since it is a numeric attribute, it makes sense to analyze the frequency values by the distribution statistics. The *mean* is 524.43 and the *standard deviation* is 546.15. 
 * **NewsDesk**: there are 12 *NewsDesk* values. The three most common are *Business* (23%), *TStyle* (11%) and *Culture* (10%). From all entries, there are 1846 missing *NewsDesk* values.
 * **SectionName**: there are 14 *SectionName* in the dataset. Some of them a related to more than one *NewsDesk* (e.g. *U.S.* is related to *National* and *Styles*). The three most frequent entries are *Business Day* (16%), *Arts* (10%) and *Opinion* (9%). There are 2300 missing values.
 * **Popularity**: is contains only 2 values: *0* and *1*. Since it is used as a dependent variable on the dataset, it does not contain any missing value. Only 16% of the records are popular.

## Concluding Remarks

In this activity the New York Time blog dataset was analyzed. Considering the goal of performing an overview of the data and computing the frequency of some items of interest, no data manipulation was performed. 

The attributes analyzed were: 
 * WordCount
 * NewsDesk
 * SectionName 
 * Popularity
 
On the next task some features should be extracted, such as week day and hour from *PubDate*. It is also possible to analyze the textual content, such as snippet and abstract, to observe the content influence on the publication popularity.